# 2. A Hands-On Introduction to the Subject
主题的动手介绍

##### Working with Datasets Libary

## Preparation for Google Colab

In [ ]:
import os
from google.colab import drive

#  挂载 google 云盘
drive.mount("/content/drive")

print(os.getcwd())  # /content

# print(os.listdir("/content/drive/MyDrive/"))

# print(os.listdir("/content/drive/MyDrive/Colab Notebooks"))

# if os.getcwd() != "/content/drive/MyDrive":
#     os.chdir("/content/drive/MyDrive")

# print(os.getcwd())

In [ ]:
# 提前将 requirements.txt 放在 google 云盘上
!pip install -r /content/drive/MyDrive/requirements.txt

In [ ]:
subdir = "ch02b"
work_path = "/content/drive/MyDrive/" + subdir
if not os.path.exists(work_path):
    os.mkdir(work_path)
os.chdir(work_path)
print(os.getcwd())

/content/drive/MyDrive/ch02b


In [4]:
!apt-get install tree && tree -a "./"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (411 kB/s)
Selecting previously unselected package tree.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
./

0 directories, 0 files


## loading a dataset

Datasets migth have several configurations. For instances, The GLUE dataset as an agregated benchmark has 10 subsets (as of writing this notebook) as: COLA, SST2, MRPC, QQP, STSB, MNLI, QNLI, RTE, WNLI and the diagnostic subset AX.

数据集可能有几种配置. 例如, GLUE 数据集作为一个聚合基准有 10 个子集(截至编写本笔记本时), 分别是: COLA、SST2、MRPC、QQP、STSB、MNLI、QNLI、RTE、WNLI 和诊断子集 AX.

To access each glue dataset, we pass two arguments where the first is **'glue'** and second is a **sub-part** of it to be chosen. Likewise, the wikipedia dataset have several configuration provided for several languages.

为了访问每个 glue 数据集, 我们传递两个参数, 第一个是 `glue`, 第二个是要选择的`子部分`. 同样, 维基百科数据集也为几种语言提供了几种配置.

Lets load 'cola' subset of GLUE as follows:

让我们加载 GLUE 的 'cola' 子集如下:

In [5]:
# It loads a dataset from the HuggingFace Hub
from datasets import load_dataset
cola = load_dataset("glue", "cola")
cola

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/251k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/37.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [7]:
!tree -a ~/.cache/huggingface/

/root/.cache/huggingface/
├── datasets
│   ├── glue
│   │   └── cola
│   │       └── 0.0.0
│   │           ├── bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c
│   │           │   ├── dataset_info.json
│   │           │   ├── glue-test.arrow
│   │           │   ├── glue-train.arrow
│   │           │   └── glue-validation.arrow
│   │           ├── bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c_builder.lock
│   │           └── bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c.incomplete_info.lock
│   └── _root_.cache_huggingface_datasets_glue_cola_0.0.0_bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c.lock
└── hub
    ├── datasets--glue
    │   ├── blobs
    │   │   ├── 274c0b2c4c5524fe09285a8b7d945fe98a1dfe7a
    │   │   ├── 2e7538afa2000e63f5343f16a758d75c452661a384208399d2035cd2fce45c33
    │   │   ├── 3c4d526b6f49f432621de43569f9ecf6af41f639baaf4a9d821b95d745def61d
    │   │   └── c14b7219a7d9f9fe3dd291fd000f6623ee413805eb108c9c49578ed50873e4ba
    │   ├── .no_exist
    │   │   └── bcdcba79d07bc864c1c254ccfcedcce5

In [8]:
cola["train"][18:22]

{'sentence': ['They drank the pub.',
  'The professor talked us into a stupor.',
  'The professor talked us.',
  'We yelled ourselves hoarse.'],
 'label': [0, 1, 0, 1],
 'idx': [18, 19, 20, 21]}

While some dataset comes with DatasetDict object, some can be of type Dataset depending on splitting condition. The CoLA dataset come with DatasetDict where we have 3 splits: train, validation, and test. Train and validation datasets include the labels as well (1: Acceptable, 0: Unacceptable), but the label values of test split are -1, which means 'no-label'.

In [9]:
cola

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [10]:
cola["train"][12]

{'sentence': 'Bill rolled out of the room.', 'label': 1, 'idx': 12}

In [11]:
cola["validation"][68]

{'sentence': 'Which report that John was incompetent did he submit?',
 'label': 0,
 'idx': 68}

In [12]:
cola["test"][20]

{'sentence': 'Has John seen Mary?', 'label': -1, 'idx': 20}

## Metadata of Datasets

In [13]:
cola["train"].info

DatasetInfo(description='', citation='', homepage='', license='', features={'sentence': Value(dtype='string', id=None), 'label': ClassLabel(names=['unacceptable', 'acceptable'], id=None), 'idx': Value(dtype='int32', id=None)}, post_processed=None, supervised_keys=None, builder_name='parquet', dataset_name='glue', config_name='cola', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=487007, num_examples=8551, shard_lengths=None, dataset_name='glue'), 'validation': SplitInfo(name='validation', num_bytes=60584, num_examples=1043, shard_lengths=None, dataset_name='glue'), 'test': SplitInfo(name='test', num_bytes=60779, num_examples=1063, shard_lengths=None, dataset_name='glue')}, download_checksums={'hf://datasets/glue@bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c/cola/train-00000-of-00001.parquet': {'num_bytes': 251124, 'checksum': None}, 'hf://datasets/glue@bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c/cola/validation-00000-of-00001.parquet': {'num_bytes': 37551, 'checksum': None},

## Loading other datasets

In [14]:
sst2 = load_dataset("glue", "sst2")
sst2

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [15]:
mrpc = load_dataset("glue", "mrpc")
mrpc

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [16]:
!tree -a ~/.cache/huggingface/

/root/.cache/huggingface/
├── datasets
│   ├── glue
│   │   ├── cola
│   │   │   └── 0.0.0
│   │   │       ├── bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c
│   │   │       │   ├── dataset_info.json
│   │   │       │   ├── glue-test.arrow
│   │   │       │   ├── glue-train.arrow
│   │   │       │   └── glue-validation.arrow
│   │   │       ├── bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c_builder.lock
│   │   │       └── bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c.incomplete_info.lock
│   │   ├── mrpc
│   │   │   └── 0.0.0
│   │   │       ├── bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c
│   │   │       │   ├── dataset_info.json
│   │   │       │   ├── glue-test.arrow
│   │   │       │   ├── glue-train.arrow
│   │   │       │   └── glue-validation.arrow
│   │   │       ├── bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c_builder.lock
│   │   │       └── bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c.incomplete_info.lock
│   │   └── sst2
│   │       └── 0.0.0
│   │           ├── bcdcba79d07bc864c1c254ccfcedcce55bcc9a8

## Listing all datasets and metrics in the hub

##### datasets

list all available datasets with `huggingface_hub.list_datasets`

##### metrics

```python
>>> import evaluate
>>> help(evaluate.list_evaluation_modules)

Help on function list_evaluation_modules in module evaluate.inspect:

list_evaluation_modules(module_type=None, include_community=True, with_details=False)
    List all evaluation modules available on the Hugging Face Hub.
    
    Args:
        module_type (`str`, *optional*, defaults to `None`):
            Type of evaluation modules to list. Has to be one of `'metric'`, `'comparison'`, or `'measurement'`. If `None`, all types are listed.
        include_community (`bool`, *optional*, defaults to `True`):
            Include community modules in the list.
        with_details (`bool`, *optional*, defaults to `False`):
            Return the full details on the metrics instead of only the ID.
    
    Returns:
        `List[Union[str, dict]]`
    
    Example:
    
    ```py
    >>> from evaluate import list_evaluation_modules
    >>> list_evaluation_modules(module_type="metric")
    ```
```

In [17]:
from huggingface_hub import list_datasets
list_datasets = list_datasets()
list_datasets
# print(len(list_datasets))
# for dataset in list_datasets[:5]:
#     print(dataset)

<generator object HfApi.list_datasets at 0x7b497cc163e0>

## Viewing the dataset as a pandas data frame

In [18]:
en_de = load_dataset("xtreme", "MLQA.en.de")

README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.89M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/336k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4517 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/512 [00:00<?, ? examples/s]

In [19]:
en_de

DatasetDict({
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4517
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 512
    })
})

In [20]:
# View dataset as a pandas data frame
import pandas as pd

pd.DataFrame(en_de["test"][0:4])

,id,title,context,question,answers
0,037e8929e7e4d2f949ffbabd10f0f860499ff7c9,Cell culture,An established or immortalized cell line has a...,Woraus besteht die Linie?,"{'answer_start': [31], 'text': ['cell']}"
1,4b36724f3cbde7c287bde512ff09194cbba7f932,Cell culture,The 19th-century English physiologist Sydney R...,Wann hat Roux etwas von seiner Medullarplatte ...,"{'answer_start': [232], 'text': ['1885']}"
2,13e58403df16d88b0e2c665953e89575704942d4,TRIPS Agreement,"After the Uruguay round, the GATT became the b...","Was muss ratifiziert werden, wenn ein Land ger...","{'answer_start': [131], 'text': ['TRIPS']}"
3,d23b5372af1de9425a4ae313c01eb80764c910d8,TRIPS Agreement,"Since TRIPS came into force, it has been subje...",Welche Teile der Welt kritisierten das TRIPS a...,"{'answer_start': [67], 'text': ['developing co..."


## Split

which split of the data to be loaded. If None by default, will return a `dict` with all splits (Train, Test, Validation or any other). If split is specified, it will return a single Dataset rather than a Dictionary

In [21]:
# Which means the first 300 examples of train plus the last 30% of validation.
cola = load_dataset("glue", "cola", split="train[:300]+validation[-30%:]")
cola

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 613
})

##### Other Split Examples

* The first 100 examples from train and validation

```python
split='train[:100]+validation[:100]
```

* 50% of train and 30 % of validation

```python
split='train[:50%]+validation[:30%]'
```

* The first 20% of train and examples in the slice 30:50 from validation

```python
split='train[:20%]+validation[30:50]'
```


## Sorting

In [22]:
cola["label"][:5]

[1, 1, 1, 1, 1]

In [23]:
cola.sort("label")["label"][:5]

[0, 0, 0, 0, 0]

In [24]:
cola.sort("label")["label"][-5:]

[1, 1, 1, 1, 1]

##  Indexing

You can also access several rows using slice notation or with a list of indices

In [25]:
cola[6, 19, 44]

{'sentence': ['Fred watered the plants flat.',
  'The professor talked us into a stupor.',
  'The trolley rumbled through the tunnel.'],
 'label': [1, 1, 1],
 'idx': [6, 19, 44]}

In [26]:
cola[42:46]

{'sentence': ['They made him to exhaustion.',
  'They made him into a monster.',
  'The trolley rumbled through the tunnel.',
  'The wagon rumbled down the road.'],
 'label': [0, 1, 1, 1],
 'idx': [42, 43, 44, 45]}

## Shuffling

In [27]:
cola[:3]

{'sentence': ["Our friends won't buy this analysis, let alone the next one we propose.",
  "One more pseudo generalization and I'm giving up.",
  "One more pseudo generalization or I'm giving up."],
 'label': [1, 1, 1],
 'idx': [0, 1, 2]}

In [28]:
cola.shuffle(seed=42)[:3]

{'sentence': ['Lou forgot the umbrella in the closet.',
  'It is the problem that he is here.',
  'I met the person who left.'],
 'label': [1, 0, 1],
 'idx': [904, 1017, 885]}

## Caching and reusability

Using cache files allows us to load large datasets by means of memory mapping if datasets fit on the drive to use a fast backend and do smart caching by saving and reusing the results of operations executed on the drive.

In [29]:
cola.cache_files

[{'filename': '/root/.cache/huggingface/datasets/glue/cola/0.0.0/bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c/glue-train.arrow'},
 {'filename': '/root/.cache/huggingface/datasets/glue/cola/0.0.0/bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c/glue-validation.arrow'}]

## filter

In [30]:
# To get 3 sentences ,including the term "kick" with Filter
cola = load_dataset("glue", "cola", split="train[:100%]+validation[-30%:]")
cola

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 8864
})

In [31]:
cola.filter(lambda s: "kick" in s["sentence"])

Filter:   0%|          | 0/8864 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 64
})

In [32]:
cola.filter(lambda s: "kick" in s["sentence"])["sentence"][:3]

['Jill kicked the ball from home plate to third base.',
 'Fred kicked the ball under the porch.',
 'Fred kicked the ball behind the tree.']

In [33]:
from pprint import pprint

# To get 3 acceptable sentences
pprint(cola.filter(lambda s: s["label"] == 1)["sentence"][:3])

Filter:   0%|          | 0/8864 [00:00<?, ? examples/s]

["Our friends won't buy this analysis, let alone the next one we propose.",
 "One more pseudo generalization and I'm giving up.",
 "One more pseudo generalization or I'm giving up."]


In [34]:
# To get 3 acceptable sentences - alternative version
cola.filter(lambda s: s["label"] == cola.features["label"].str2int("acceptable"))[
    "sentence"
][:3]

Filter:   0%|          | 0/8864 [00:00<?, ? examples/s]

["Our friends won't buy this analysis, let alone the next one we propose.",
 "One more pseudo generalization and I'm giving up.",
 "One more pseudo generalization or I'm giving up."]

## map

`datasets.Dataset.map()` function iterates over the dataset applying a processing function to each examples in a dataset and modifies the content of the samples.

In [35]:
# E.g. adding new features
cola_new = cola.map(lambda e: {"len": len(e["sentence"])})

Map:   0%|          | 0/8864 [00:00<?, ? examples/s]

In [36]:
cola_new

Dataset({
    features: ['sentence', 'label', 'idx', 'len'],
    num_rows: 8864
})

In [37]:
pprint(cola_new[0:3])

{'idx': [0, 1, 2],
 'label': [1, 1, 1],
 'len': [71, 49, 48],
 'sentence': ["Our friends won't buy this analysis, let alone the next one we "
              'propose.',
              "One more pseudo generalization and I'm giving up.",
              "One more pseudo generalization or I'm giving up."]}


In [38]:
cola_cut = cola_new.map(lambda e: {"sentence": e["sentence"][:20]})

Map:   0%|          | 0/8864 [00:00<?, ? examples/s]

In [39]:
pprint(cola_cut[:3])

{'idx': [0, 1, 2],
 'label': [1, 1, 1],
 'len': [71, 49, 48],
 'sentence': ["Our friends won't bu",
              'One more pseudo gene',
              'One more pseudo gene']}


In [40]:
!tree -a ~/.cache/huggingface

/root/.cache/huggingface
├── datasets
│   ├── glue
│   │   ├── cola
│   │   │   └── 0.0.0
│   │   │       ├── bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c
│   │   │       │   ├── cache-3f3b9afccad65ffb.arrow
│   │   │       │   ├── cache-434e67726f2b20e9.arrow
│   │   │       │   ├── cache-5d0f7797b9e79edc.arrow
│   │   │       │   ├── cache-5e7ccdce59c20329.arrow
│   │   │       │   ├── cache-794b443e391da5c3.arrow
│   │   │       │   ├── cache-7d9b7f7d58672a93.arrow
│   │   │       │   ├── cache-94970668003b3cd8.arrow
│   │   │       │   ├── dataset_info.json
│   │   │       │   ├── glue-test.arrow
│   │   │       │   ├── glue-train.arrow
│   │   │       │   └── glue-validation.arrow
│   │   │       ├── bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c_builder.lock
│   │   │       └── bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c.incomplete_info.lock
│   │   ├── mrpc
│   │   │   └── 0.0.0
│   │   │       ├── bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c
│   │   │       │   ├── dataset_info.json
│   │   │  

## Working with Local Files

In [41]:
# To load a dataset from local files CSV, TXT, JSON, a generic loading scripts are provided
# under data folder there are the files[a.csv, b.csv, c.csv], some random part of SST-2 dataset
from datasets import load_dataset

data1 = load_dataset("csv", data_files="./data/a.csv", delimiter="\t")
print("data1:", data1)

data2 = load_dataset(
    "csv", data_files=["./data/a.csv", "./data/b.csv", "./data/c.csv"], delimiter="\t"
)
print("data2:", data2)

data3 = load_dataset(
    "csv",
    data_files={"train": ["./data/a.csv", "./data/b.csv"], "test": ["./data/c.csv"]},
    delimiter="\t",
)
print("data3:", data3)

Generating train split: 0 examples [00:00, ? examples/s]

data1: DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 99
    })
})


Generating train split: 0 examples [00:00, ? examples/s]

data2: DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 299
    })
})


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

data3: DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 199
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 100
    })
})


In [42]:
import pandas as pd

pd.DataFrame(data1["train"][:3])

,sentence,label
0,hide new secretions from the parental units,0
1,"contains no wit , only labored gags",0
2,that loves its characters and communicates som...,1


In [43]:
import pandas as pd

pd.DataFrame(data2["train"][:3])

,sentence,label
0,hide new secretions from the parental units,0
1,"contains no wit , only labored gags",0
2,that loves its characters and communicates som...,1


In [44]:
pd.DataFrame(data3["test"][:3])

,sentence,label
0,inane and awful,0
1,told in scattered fashion,0
2,takes chances that are bold by studio standards,1


get the files in other format

```python
data_json = load_dataset('json', data_files='a.json')
data_text = load_dataset('text', data_files='a.txt')
```

## Preparing the data for model training

In [45]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

DistilBertTokenizer(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [46]:
encoded_data1 = data1.map(
    lambda e: tokenizer(e["sentence"]), batched=True, batch_size=1000
)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [47]:
data1

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 99
    })
})

In [48]:
encoded_data1

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 99
    })
})

In [49]:
pprint(data1["train"][0])

{'label': 0, 'sentence': 'hide new secretions from the parental units '}


In [50]:
pprint(encoded_data1["train"][0])

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102],
 'label': 0,
 'sentence': 'hide new secretions from the parental units '}


In [51]:
encoded_data3 = data3.map(
    lambda e: tokenizer(e["sentence"], padding=True, truncation=True, max_length=12),
    batched=True,
    batch_size=1000,
)

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [52]:
data3

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 199
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 100
    })
})

In [53]:
encoded_data3

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 199
    })
    test: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 100
    })
})

In [54]:
pprint(data3["test"][12])

{'label': 0, 'sentence': 'an extremely unpleasant film . '}


In [55]:
pprint(encoded_data3["test"][12])

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 'input_ids': [101, 2019, 5186, 16010, 2143, 1012, 102, 0, 0, 0, 0, 0],
 'label': 0,
 'sentence': 'an extremely unpleasant film . '}
